In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
from snake import SnakeGame
from agent import A2CAgent

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle

loss_hist = {1: [], 2: []}


def train_agents(episodes):
    env = SnakeGame(10, 10, max_steps = 20000)
    agent = A2CAgent()

    batch_size = 64

    rewards = []
    for e in range(episodes):
        state = env.reset()
        acc_reward = 0

        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            acc_reward += reward

            state = next_state
            if done:
                break

        agent.replay(batch_size)
        rewards.append(acc_reward)

        if e % 5000 == 0:
            print(f"episode={e}")

    return agent, rewards


agent, rewards = train_agents(500000)

/Users/maurits/micromamba/envs/develop/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


episode=0
episode=5000
episode=10000
episode=15000
episode=20000
episode=25000
episode=30000
episode=35000
episode=40000
episode=45000
episode=50000
episode=55000
episode=60000
episode=65000
episode=70000
episode=75000
episode=80000
episode=85000
episode=90000
episode=95000
episode=100000
episode=105000
episode=110000
episode=115000
episode=120000
episode=125000
episode=130000
episode=135000
episode=140000
episode=145000
episode=150000
episode=155000
episode=160000
episode=165000
episode=170000
episode=175000
episode=180000
episode=185000
episode=190000
episode=195000
episode=200000
episode=205000
episode=210000
episode=215000
episode=220000
episode=225000
episode=230000
episode=235000
episode=240000
episode=245000
episode=250000
episode=255000
episode=260000
episode=265000
episode=270000
episode=275000
episode=280000
episode=285000
episode=290000
episode=295000
episode=300000
episode=305000
episode=310000
episode=315000
episode=320000
episode=325000
episode=330000
episode=335000
episo

In [5]:
import torch

agent.save("./agent_10x10.state")
torch.save(rewards, "./rewards_10x10.state")

Agent saved to agent_10x10.state
